# Import common libraries

In [1]:
import pandas as pd
import numpy as np

# Get the initial text

In [2]:
s =  'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

# Create Table as Dataframe

In [3]:
# Parse over the raw data
from io import StringIO
df = pd.read_csv(StringIO(s), delimiter=';', lineterminator='\n')

In [4]:
df.head(15)

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055.0,london_MONTreal


## 1. FlightCodes column

Assumption => There is at least one Non-Null values.

In [5]:
# Get index of first Not NULL value
idx_nn = df['FlightCodes'].first_valid_index()
# Get the first Not NULL value
value_nn = df.loc[idx_nn, 'FlightCodes']

# Take the valid index and value and interpolate it for the entire column
df['FlightCodes'] = df.apply(lambda row : ((row.name - idx_nn)*10) + value_nn, axis=1).astype(int)

df.head()

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork
1,<Air France> (12),[],20025,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055,london_MONTreal


## 2. To_From column

In [6]:
# Transform the whole string to uppercase
df['To_From'] = df['To_From'].str.upper()
# Split on '_' to create two columns respectively
df[['To', 'From']] = df['To_From'].str.split('_', 1, expand=True)
# Drop To_from column
df.drop(columns=["To_From"], inplace = True)

df.head()

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015,WATERLOO,NEWYORK
1,<Air France> (12),[],20025,MONTREAL,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12. Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,""".\.Lufthansa.\.""","[12, 33]",20055,LONDON,MONTREAL


## 3. Airline Code column

Assumption => We only need to remove the punctuation and not the numbers

In [7]:
# Remove all punctuation except the space
df['Airline Code'] = df['Airline Code'].str.replace('[^a-zA-Z0-9\s]', '')
# Remove all the leading and trailing whitespaces
df['Airline Code'] = df['Airline Code'].str.strip()

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France 12,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12 Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL
